In [34]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn import ensemble, metrics, model_selection, preprocessing
#from utils import plot_decision

In [2]:
df = pd.read_json('/Users/kavetiraviteja/Downloads/mixed_features_gold_tagged_cleared_better/part-00000-520c4808-b1e1-4eef-9bc5-41caad5cc443-c000.json',lines=True)
df.head()

,articleId,articleLength,companyId,countOfDistinctIndustry,countOfOccurence,countOfOccurenceOfAliases,countOfOccurenceOfNamePreserveList,countOfSignificantStrings,countOfUrlMatch,countryMatchByOverlapScore,...,isProminentByParent,isPublic,isSingleCompany,isTickerMatch,isUrlMatched,mentionMatched,personScore,stateMatchByOverlapScore,sumOfIndustry,w2vScore
0,33441345460,4105,18353195,0,0,0,2,2,0,0.0,...,False,False,False,False,False,hunton and williams,-1000.0,-1000.0,0,0.0
1,33441345460,4105,10261568,2,3,3,3,3,0,0.5,...,True,False,True,False,False,houston office,-1000.0,0.7,21,0.0
2,33441345460,4105,51217888,0,0,0,6,6,0,0.0,...,False,False,False,False,False,shearman and sterling,0.9,-1000.0,0,0.0
3,33441345460,4105,18351925,0,0,0,2,2,0,0.5,...,False,False,False,False,False,hunton and williams,-1000.0,0.0,0,0.0
4,33441345460,4105,18348347,0,3,3,3,3,0,0.0,...,False,False,False,False,False,baker botts,0.9,-1000.0,0,0.0


In [23]:
# select features
# df['species_label'], _ = pd.factorize(df['iris'])
y = df['isGold']
X = df[['w2vScore','personScore','sumOfIndustry',
        'countOfOccurenceOfAliases','countOfSignificantStrings',
        'countryMatchByOverlapScore','directCityMatchScore','directCountryMatchScore',
        'directStateMatchScore','exactMatch','isIndustry','isProminentByItself','isPublic'
        ,'isProminentByParent','isSingleCompany','isTickerMatch','isUrlMatched','stateMatchByOverlapScore'
       ]]

In [24]:
# split data randomly into 70% training and 30% test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=0)

In [25]:
# train the random forest model using 10 decision trees and 2 CPU cores
rf = ensemble.RandomForestClassifier(criterion='entropy', n_estimators=15, random_state=1, n_jobs=2)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=2,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [26]:

# use the model to make predictions with the test data
y_pred = rf.predict(X_test)

In [27]:
X1_test=df.copy()
X1_test['y_test']=y_test
X1_test['y_pred']=pd.DataFrame(y_pred)
print (X1_test.shape)

(5922, 33)


In [28]:
# how did our model perform?
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

X1_test.to_csv('/Users/kavetiraviteja/Downloads/rf_predict.csv', sep=',')

Misclassified samples: 176
Accuracy: 0.90


In [29]:
print(dict(zip(X.columns, rf.feature_importances_)))

{'w2vScore': 0.09316206670039108, 'personScore': 0.16314694665547255, 'sumOfIndustry': 0.10362907814462492, 'countOfOccurenceOfAliases': 0.04363510695610942, 'countOfSignificantStrings': 0.048061677582350776, 'countryMatchByOverlapScore': 0.016764778307693493, 'directCityMatchScore': 0.027566161831099334, 'directCountryMatchScore': 0.0212339557277059, 'directStateMatchScore': 0.028987539836185284, 'exactMatch': 0.02244523915320141, 'isIndustry': 0.0021371493716033784, 'isProminentByItself': 0.2082872951847922, 'isPublic': 0.025404689596872306, 'isProminentByParent': 0.054603827603226705, 'isSingleCompany': 0.0877383258546056, 'isTickerMatch': 0.017242621774768544, 'isUrlMatched': 0.009639545246512131, 'stateMatchByOverlapScore': 0.026313994472785027}


In [30]:
d = dict(zip(X.columns, rf.feature_importances_))

In [31]:
for w in sorted(d, key=d.get, reverse=True):
    print(w, d[w])

isProminentByItself 0.2082872951847922
personScore 0.16314694665547255
sumOfIndustry 0.10362907814462492
w2vScore 0.09316206670039108
isSingleCompany 0.0877383258546056
isProminentByParent 0.054603827603226705
countOfSignificantStrings 0.048061677582350776
countOfOccurenceOfAliases 0.04363510695610942
directStateMatchScore 0.028987539836185284
directCityMatchScore 0.027566161831099334
stateMatchByOverlapScore 0.026313994472785027
isPublic 0.025404689596872306
exactMatch 0.02244523915320141
directCountryMatchScore 0.0212339557277059
isTickerMatch 0.017242621774768544
countryMatchByOverlapScore 0.016764778307693493
isUrlMatched 0.009639545246512131
isIndustry 0.0021371493716033784
